## Imports

In [4]:
import warnings
from itertools import product
import glob
from datetime import datetime
from datetime import timedelta
import numpy as np
import pandas as pd
import xarray as xr

import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.path as mpath
import cartopy
import cartopy.crs as ccrs
import cartopy.feature
import cartopy.feature as cfeature
import cartopy.io.shapereader as shpreader
import cartopy.feature as cf
import shapely.geometry as sgeom

from sklearn.decomposition import PCA
from scipy import stats
from sklearn.cluster import KMeans
from sklearn import metrics
from scipy.spatial.distance import cdist
import xskillscore as xs

import som_analysis
import cluster_analysis
import narm_analysis

## functions

In [6]:
def add_insetmap(axes_extent, map_extent, lons, lats, temp_data, 
                 vmin, vmax, cmap='coolwarm'):

    use_projection = ccrs.Mercator()     # preserve shape well
    geodetic = ccrs.Geodetic(globe=ccrs.Globe(datum='WGS84'))

    sub_ax = plt.axes(axes_extent, projection=use_projection)  # normal units

    sub_ax.set_extent(map_extent, geodetic)  # map extents
    sub_ax.coastlines(linewidth=0.35, zorder=10)

    sub_ax.pcolormesh(lons, lats, temp_data, transform=ccrs.PlateCarree(),
                      vmin=vmin, vmax=vmax, cmap=cmap)

    extent_box = sgeom.box(
        map_extent[0], map_extent[2], map_extent[1], map_extent[3])
    sub_ax.add_geometries(
        [extent_box], ccrs.PlateCarree(), color='none', linewidth=0.05)

In [7]:
def get_cold_indx(ds, mo_init=9, mo_end=2):
    """
    Extract indices for cold season.
    Grabbing Sept thru February init, for Oct thru March predictions.
    """
    dt_array = pd.to_datetime(ds['time'])
    return xr.where((dt_array.month >= mo_init) | (
        dt_array.month <= mo_end), True, False)

## open and preprocess data

In [8]:
# region for clustering
lat0 = 10
lat1 = 70
lon0 = -150
lon1 = -40

# open era5 data and slice
ds_era5 = narm_analysis.era5_z500(lat0=lat0, lat1=lat1, lon0=lon0, lon1=lon1)

# era5 anomalies
ds_era5_anom = narm_analysis.era5_climo_wrs(
    ds_era5, rolling_days=5, variable='clim')

# restructure era5 array for machine learning training (SONDJFM)
ds_era5_anom = ds_era5_anom[get_cold_indx(
    ds_era5_anom, mo_init=10, mo_end=3), ...]
ds_era5_train = ds_era5_anom.stack(
    flat=('lat', 'lon')).transpose('time', 'flat').values

## pca and kmeans with era5

In [9]:
# create pca object
pca_obj = PCA(12, whiten=True)

# fit pca with era5
pca_obj = pca_obj.fit(ds_era5_train)

# transform era5 data with pca
ds_era5_train = pca_obj.transform(ds_era5_train)

print(f'Variance explained: {pca_obj.explained_variance_ratio_ * 100}')
print(
f'Cumulative sum of variance explained for EOF1 and EOF2: {np.cumsum(pca_obj.explained_variance_ratio_) * 100}'
)

# train kmeans
k_means = KMeans(n_clusters=4,
                 init='k-means++',
                 n_init=10000,
                 max_iter=300,
                 tol=0.0001,
                 verbose=0,
                 random_state=0).fit(ds_era5_train)

print(f'inertia: {k_means.inertia_}')

Variance explained: [25.95315607 17.65410568 11.94871708  9.0784389   7.98100848  6.14181738
  4.32605934  2.61658689  2.22642929  2.17049559  1.49813958  1.22541708]
Cumulative sum of variance explained for EOF1 and EOF2: [25.95315607 43.60726175 55.55597883 64.63441774 72.61542622 78.7572436
 83.08330294 85.69988983 87.92631912 90.09681471 91.59495429 92.82037136]
inertia: 39379.205381011976


## load data with lead time bias corrected anomalies

In [10]:
# era5 data
z500_era5, z500_era5_dt = som_analysis.open_era5_files(
    variable='z500', return_time=True,
    lat0=lat0, lat1=lat1, lon0=lon0, lon1=lon1,
    leadday0=0, leadday1=42, rolldays=5)

# cesm data
z500_cesm, z500_cesm_dt = som_analysis.open_cesm_files(
    variable='zg_500', return_time=True,
    lat0=lat0, lat1=lat1, lon0=lon0, lon1=lon1,
    leadday0=0, leadday1=42, rolldays=5)

# restructure arrays
z500_standard_era5 = z500_era5.stack(
    new=('time', 'lead'), flat=('lat', 'lon')).transpose('new', 'flat')
z500_standard_cesm = z500_cesm.stack(
    new=('time', 'lead'), flat=('lat', 'lon')).transpose('new', 'flat')

## composites of the weather types/regimes

In [11]:
# grab cluster indices

z500_era5_tmp_1, z500_era5_tmp_2, z500_era5_tmp_3, z500_era5_tmp_4 = cluster_analysis.composite_clusters_indx(
    z500_standard_era5, k_means, pca_obj, use_pca=True)

z500_cesm_tmp_1, z500_cesm_tmp_2, z500_cesm_tmp_3, z500_cesm_tmp_4 = cluster_analysis.composite_clusters_indx(
    z500_standard_cesm, k_means, pca_obj, use_pca=True)

## temperature anomalies

In [12]:
lat0_tmp = 10
lat1_tmp = 75
lon0_tmp = -165
lon1_tmp = -40

# temperature

# noaa data
t2m_noaa, _ = som_analysis.open_noaa_files(
    variable='temp', return_time=True,
    lat0=lat0_tmp, lat1=lat1_tmp, lon0=lon0_tmp, lon1=lon1_tmp,
    leadday0=0, leadday1=42, rolldays=1,)

mask = xr.where(~np.isnan(t2m_noaa.isel(time=0, lead=0)), 1.0, np.nan)

# era5 data
t2m_era5, _ = som_analysis.open_era5_files(
    variable='temp', return_time=True,
    lat0=lat0_tmp, lat1=lat1_tmp, lon0=lon0_tmp, lon1=lon1_tmp,
    leadday0=0, leadday1=42, rolldays=1,)

t2m_era5 = t2m_era5.where(mask == 1.0)

# cesm data
t2m_cesm, _ = som_analysis.open_cesm_files(
    variable='tas_2m', return_time=True,
    lat0=lat0_tmp, lat1=lat1_tmp, lon0=lon0_tmp, lon1=lon1_tmp,
    leadday0=0, leadday1=42, rolldays=1,)

t2m_cesm = t2m_cesm.where(mask == 1.0)

In [13]:
t2m_noaa_01 = t2m_noaa.stack(new=('time', 'lead')).transpose(
    'new', 'lat', 'lon')[z500_era5_tmp_1, :, :]
t2m_noaa_02 = t2m_noaa.stack(new=('time', 'lead')).transpose(
    'new', 'lat', 'lon')[z500_era5_tmp_2, :, :]
t2m_noaa_03 = t2m_noaa.stack(new=('time', 'lead')).transpose(
    'new', 'lat', 'lon')[z500_era5_tmp_3, :, :]
t2m_noaa_04 = t2m_noaa.stack(new=('time', 'lead')).transpose(
    'new', 'lat', 'lon')[z500_era5_tmp_4, :, :]

t2m_era5_01 = t2m_era5.stack(new=('time', 'lead')).transpose(
    'new', 'lat', 'lon')[z500_era5_tmp_1, :, :]
t2m_era5_02 = t2m_era5.stack(new=('time', 'lead')).transpose(
    'new', 'lat', 'lon')[z500_era5_tmp_2, :, :]
t2m_era5_03 = t2m_era5.stack(new=('time', 'lead')).transpose(
    'new', 'lat', 'lon')[z500_era5_tmp_3, :, :]
t2m_era5_04 = t2m_era5.stack(new=('time', 'lead')).transpose(
    'new', 'lat', 'lon')[z500_era5_tmp_4, :, :]

t2m_cesm_01 = t2m_cesm.stack(new=('time', 'lead')).transpose(
    'new', 'lat', 'lon')[z500_cesm_tmp_1, :, :]
t2m_cesm_02 = t2m_cesm.stack(new=('time', 'lead')).transpose(
    'new', 'lat', 'lon')[z500_cesm_tmp_2, :, :]
t2m_cesm_03 = t2m_cesm.stack(new=('time', 'lead')).transpose(
    'new', 'lat', 'lon')[z500_cesm_tmp_3, :, :]
t2m_cesm_04 = t2m_cesm.stack(new=('time', 'lead')).transpose(
    'new', 'lat', 'lon')[z500_cesm_tmp_4, :, :]

## bootstrap

In [14]:
firstday = 1
seconday = 14

lons = t2m_noaa_01.lon.values
lats = t2m_noaa_01.lat.values

boot_num_init_ = 1000
boot_num_iter_ = 10000

## noaa bootstrap

In [ ]:
tmp_all = t2m_noaa.isel(lead=slice(firstday, seconday)).stack(
    new=('time', 'lead')).transpose('new', 'lat', 'lon').values

In [ ]:
tmp_data = t2m_noaa_01.unstack('new').isel(lead=slice(firstday, seconday)).stack(
    new=('lead', 'time')).transpose('new', 'lat', 'lon').values

for ind in range(boot_num_init_, boot_num_iter_):

    np.random.seed(ind + 1)
    rand_indx = [np.random.choice(tmp_all.shape[0]) for i in range(tmp_data.shape[0])]
    boot_ = np.nanmean(tmp_all[rand_indx, ...], axis=0)

    xr.Dataset(
        data_vars=dict(
            iteration=(["lat", "lon"], boot_),
        ),
        coords=dict(
            lon=(["lon"], lons),
            lat=(["lat"], lats),
        ),
        attrs=dict(description="For bootstrap confidence intervals."),
    ).to_netcdf(
        f'/glade/scratch/molina/s2s/bootstrap/t2m_ncpc_wr1_week12/t2m_ncpc_boot_{ind + 1}.nc')

In [12]:
tmp_data = t2m_noaa_02.unstack('new').isel(lead=slice(firstday, seconday)).stack(
    new=('lead', 'time')).transpose('new', 'lat', 'lon').values

for ind in range(7200, boot_num_iter_):

    np.random.seed(ind + 1)
    rand_indx = [np.random.choice(tmp_all.shape[0]) for i in range(tmp_data.shape[0])]
    boot_ = np.nanmean(tmp_all[rand_indx, ...], axis=0)

    xr.Dataset(
        data_vars=dict(
            iteration=(["lat", "lon"], boot_),
        ),
        coords=dict(
            lon=(["lon"], lons),
            lat=(["lat"], lats),
        ),
        attrs=dict(description="For bootstrap confidence intervals."),
    ).to_netcdf(
        f'/glade/scratch/molina/s2s/bootstrap/t2m_ncpc_wr2_week12/t2m_ncpc_boot_{ind + 1}.nc')

In [13]:
tmp_data = t2m_noaa_03.unstack('new').isel(lead=slice(firstday, seconday)).stack(
    new=('lead', 'time')).transpose('new', 'lat', 'lon').values

for ind in range(boot_num_init_, boot_num_iter_):

    np.random.seed(ind + 1)
    rand_indx = [np.random.choice(tmp_all.shape[0]) for i in range(tmp_data.shape[0])]
    boot_ = np.nanmean(tmp_all[rand_indx, ...], axis=0)

    xr.Dataset(
        data_vars=dict(
            iteration=(["lat", "lon"], boot_),
        ),
        coords=dict(
            lon=(["lon"], lons),
            lat=(["lat"], lats),
        ),
        attrs=dict(description="For bootstrap confidence intervals."),
    ).to_netcdf(
        f'/glade/scratch/molina/s2s/bootstrap/t2m_ncpc_wr3_week12/t2m_ncpc_boot_{ind + 1}.nc')

/glade/scratch/molina/ipykernel_189713/2060033009.py:8: RuntimeWarning: Mean of empty slice
  boot_ = np.nanmean(tmp_all[rand_indx, ...], axis=0)


In [14]:
tmp_data = t2m_noaa_04.unstack('new').isel(lead=slice(firstday, seconday)).stack(
    new=('lead', 'time')).transpose('new', 'lat', 'lon').values

for ind in range(boot_num_init_, boot_num_iter_):

    np.random.seed(ind + 1)
    rand_indx = [np.random.choice(tmp_all.shape[0]) for i in range(tmp_data.shape[0])]
    boot_ = np.nanmean(tmp_all[rand_indx, ...], axis=0)

    xr.Dataset(
        data_vars=dict(
            iteration=(["lat", "lon"], boot_),
        ),
        coords=dict(
            lon=(["lon"], lons),
            lat=(["lat"], lats),
        ),
        attrs=dict(description="For bootstrap confidence intervals."),
    ).to_netcdf(
        f'/glade/scratch/molina/s2s/bootstrap/t2m_ncpc_wr4_week12/t2m_ncpc_boot_{ind + 1}.nc')

/glade/scratch/molina/ipykernel_189713/4291095622.py:8: RuntimeWarning: Mean of empty slice
  boot_ = np.nanmean(tmp_all[rand_indx, ...], axis=0)


## era5 bootstrap

In [11]:
tmp_all = t2m_era5.isel(lead=slice(firstday, seconday)).stack(
    new=('time', 'lead')).transpose('new', 'lat', 'lon').values

In [16]:
tmp_data = t2m_era5_01.unstack('new').isel(lead=slice(firstday, seconday)).stack(
    new=('lead', 'time')).transpose('new', 'lat', 'lon').values

for ind in range(boot_num_init_, boot_num_iter_):

    np.random.seed(ind + 1)
    rand_indx = [np.random.choice(tmp_all.shape[0]) for i in range(tmp_data.shape[0])]
    boot_ = np.nanmean(tmp_all[rand_indx, ...], axis=0)

    xr.Dataset(
        data_vars=dict(
            iteration=(["lat", "lon"], boot_),
        ),
        coords=dict(
            lon=(["lon"], lons),
            lat=(["lat"], lats),
        ),
        attrs=dict(description="For bootstrap confidence intervals."),
    ).to_netcdf(
        f'/glade/scratch/molina/s2s/bootstrap/t2m_era5_wr1_week12/t2m_era5_boot_{ind + 1}.nc')

/glade/scratch/molina/ipykernel_189713/1541693190.py:8: RuntimeWarning: Mean of empty slice
  boot_ = np.nanmean(tmp_all[rand_indx, ...], axis=0)


In [17]:
tmp_data = t2m_era5_02.unstack('new').isel(lead=slice(firstday, seconday)).stack(
    new=('lead', 'time')).transpose('new', 'lat', 'lon').values

for ind in range(boot_num_init_, boot_num_iter_):

    np.random.seed(ind + 1)
    rand_indx = [np.random.choice(tmp_all.shape[0]) for i in range(tmp_data.shape[0])]
    boot_ = np.nanmean(tmp_all[rand_indx, ...], axis=0)

    xr.Dataset(
        data_vars=dict(
            iteration=(["lat", "lon"], boot_),
        ),
        coords=dict(
            lon=(["lon"], lons),
            lat=(["lat"], lats),
        ),
        attrs=dict(description="For bootstrap confidence intervals."),
    ).to_netcdf(
        f'/glade/scratch/molina/s2s/bootstrap/t2m_era5_wr2_week12/t2m_era5_boot_{ind + 1}.nc')

/glade/scratch/molina/ipykernel_189713/582870324.py:8: RuntimeWarning: Mean of empty slice
  boot_ = np.nanmean(tmp_all[rand_indx, ...], axis=0)


In [12]:
tmp_data = t2m_era5_03.unstack('new').isel(lead=slice(firstday, seconday)).stack(
    new=('lead', 'time')).transpose('new', 'lat', 'lon').values

for ind in range(5702, boot_num_iter_):

    np.random.seed(ind + 1)
    rand_indx = [np.random.choice(tmp_all.shape[0]) for i in range(tmp_data.shape[0])]
    boot_ = np.nanmean(tmp_all[rand_indx, ...], axis=0)

    xr.Dataset(
        data_vars=dict(
            iteration=(["lat", "lon"], boot_),
        ),
        coords=dict(
            lon=(["lon"], lons),
            lat=(["lat"], lats),
        ),
        attrs=dict(description="For bootstrap confidence intervals."),
    ).to_netcdf(
        f'/glade/scratch/molina/s2s/bootstrap/t2m_era5_wr3_week12/t2m_era5_boot_{ind + 1}.nc')

/glade/scratch/molina/ipykernel_136319/359358041.py:8: RuntimeWarning: Mean of empty slice
  boot_ = np.nanmean(tmp_all[rand_indx, ...], axis=0)


In [ ]:
tmp_data = t2m_era5_04.unstack('new').isel(lead=slice(firstday, seconday)).stack(
    new=('lead', 'time')).transpose('new', 'lat', 'lon').values

for ind in range(boot_num_init_, boot_num_iter_):

    np.random.seed(ind + 1)
    rand_indx = [np.random.choice(tmp_all.shape[0]) for i in range(tmp_data.shape[0])]
    boot_ = np.nanmean(tmp_all[rand_indx, ...], axis=0)

    xr.Dataset(
        data_vars=dict(
            iteration=(["lat", "lon"], boot_),
        ),
        coords=dict(
            lon=(["lon"], lons),
            lat=(["lat"], lats),
        ),
        attrs=dict(description="For bootstrap confidence intervals."),
    ).to_netcdf(
        f'/glade/scratch/molina/s2s/bootstrap/t2m_era5_wr4_week12/t2m_era5_boot_{ind + 1}.nc')

/glade/scratch/molina/ipykernel_136319/2611253228.py:8: RuntimeWarning: Mean of empty slice
  boot_ = np.nanmean(tmp_all[rand_indx, ...], axis=0)


## cesm bootstrap

In [15]:
tmp_all = t2m_cesm.isel(lead=slice(firstday, seconday)).stack(
    new=('time', 'lead')).transpose('new', 'lat', 'lon').values

In [ ]:
tmp_data = t2m_cesm_01.unstack('new').isel(lead=slice(firstday,seconday)).stack(
    new=('lead','time')).transpose('new','lat','lon').values

for ind in range(boot_num_init_, boot_num_iter_):

    np.random.seed(ind + 1)
    rand_indx = [np.random.choice(tmp_all.shape[0]) for i in range(tmp_data.shape[0])]
    boot_ = np.nanmean(tmp_all[rand_indx,...], axis=0)
    
    xr.Dataset(
        data_vars=dict(
            iteration=(["lat", "lon"], boot_),
        ),
        coords=dict(
            lon=(["lon"], lons),
            lat=(["lat"], lats),
        ),
        attrs=dict(description="For bootstrap confidence intervals."),
    ).to_netcdf(
        f'/glade/scratch/molina/s2s/bootstrap/t2m_cesm_wr1_week12/t2m_cesm_boot_{ind + 1}.nc')

In [ ]:
tmp_data = t2m_cesm_02.unstack('new').isel(lead=slice(firstday,seconday)).stack(
    new=('lead','time')).transpose('new','lat','lon').values

for ind in range(boot_num_init_, boot_num_iter_):

    np.random.seed(ind + 1)
    rand_indx = [np.random.choice(tmp_all.shape[0]) for i in range(tmp_data.shape[0])]
    boot_ = np.nanmean(tmp_all[rand_indx,...], axis=0)
    
    xr.Dataset(
        data_vars=dict(
            iteration=(["lat", "lon"], boot_),
        ),
        coords=dict(
            lon=(["lon"], lons),
            lat=(["lat"], lats),
        ),
        attrs=dict(description="For bootstrap confidence intervals."),
    ).to_netcdf(
        f'/glade/scratch/molina/s2s/bootstrap/t2m_cesm_wr2_week12/t2m_cesm_boot_{ind + 1}.nc')

In [ ]:
tmp_data = t2m_cesm_03.unstack('new').isel(lead=slice(firstday,seconday)).stack(
    new=('lead','time')).transpose('new','lat','lon').values

for ind in range(boot_num_init_, boot_num_iter_):

    np.random.seed(ind + 1)
    rand_indx = [np.random.choice(tmp_all.shape[0]) for i in range(tmp_data.shape[0])]
    boot_ = np.nanmean(tmp_all[rand_indx,...], axis=0)
    
    xr.Dataset(
        data_vars=dict(
            iteration=(["lat", "lon"], boot_),
        ),
        coords=dict(
            lon=(["lon"], lons),
            lat=(["lat"], lats),
        ),
        attrs=dict(description="For bootstrap confidence intervals."),
    ).to_netcdf(
        f'/glade/scratch/molina/s2s/bootstrap/t2m_cesm_wr3_week12/t2m_cesm_boot_{ind + 1}.nc')

In [16]:
tmp_data = t2m_cesm_04.unstack('new').isel(lead=slice(firstday, seconday)).stack(
    new=('lead', 'time')).transpose('new', 'lat', 'lon').values

for ind in range(3590, boot_num_iter_):

    np.random.seed(ind + 1)
    rand_indx = [np.random.choice(tmp_all.shape[0]) for i in range(tmp_data.shape[0])]
    boot_ = np.nanmean(tmp_all[rand_indx, ...], axis=0)

    xr.Dataset(
        data_vars=dict(
            iteration=(["lat", "lon"], boot_),
        ),
        coords=dict(
            lon=(["lon"], lons),
            lat=(["lat"], lats),
        ),
        attrs=dict(description="For bootstrap confidence intervals."),
    ).to_netcdf(
        f'/glade/scratch/molina/s2s/bootstrap/t2m_cesm_wr4_week12/t2m_cesm_boot_{ind + 1}.nc')

/glade/scratch/molina/ipykernel_200178/2049057487.py:8: RuntimeWarning: Mean of empty slice
  boot_ = np.nanmean(tmp_all[rand_indx, ...], axis=0)


## compute bootstrap percentiles

In [17]:
lev_1 = 0.025
lev_2 = 0.975
lev_3 = 0.005
lev_4 = 0.995

# ncpc

tmp_ncpc_wr1 = xr.open_mfdataset(
    '/glade/scratch/molina/s2s/bootstrap/t2m_ncpc_wr1_week12/t2m_ncpc_boot_*.nc',
    combine='nested', concat_dim='iter').chunk(
    dict(iter=-1)).quantile([lev_1, lev_2, lev_3, lev_4], dim='iter', skipna=True)

tmp_ncpc_wr2 = xr.open_mfdataset(
    '/glade/scratch/molina/s2s/bootstrap/t2m_ncpc_wr2_week12/t2m_ncpc_boot_*.nc',
    combine='nested', concat_dim='iter').chunk(
    dict(iter=-1)).quantile([lev_1, lev_2, lev_3, lev_4], dim='iter', skipna=True)

tmp_ncpc_wr3 = xr.open_mfdataset(
    '/glade/scratch/molina/s2s/bootstrap/t2m_ncpc_wr3_week12/t2m_ncpc_boot_*.nc',
    combine='nested', concat_dim='iter').chunk(
    dict(iter=-1)).quantile([lev_1, lev_2, lev_3, lev_4], dim='iter', skipna=True)

tmp_ncpc_wr4 = xr.open_mfdataset(
    '/glade/scratch/molina/s2s/bootstrap/t2m_ncpc_wr4_week12/t2m_ncpc_boot_*.nc',
    combine='nested', concat_dim='iter').chunk(
    dict(iter=-1)).quantile([lev_1, lev_2, lev_3, lev_4], dim='iter', skipna=True)

# era5

tmp_era5_wr1 = xr.open_mfdataset(
    '/glade/scratch/molina/s2s/bootstrap/t2m_era5_wr1_week12/t2m_era5_boot_*.nc',
    combine='nested', concat_dim='iter').chunk(
    dict(iter=-1)).quantile([lev_1, lev_2, lev_3, lev_4], dim='iter', skipna=True)

tmp_era5_wr2 = xr.open_mfdataset(
    '/glade/scratch/molina/s2s/bootstrap/t2m_era5_wr2_week12/t2m_era5_boot_*.nc',
    combine='nested', concat_dim='iter').chunk(
    dict(iter=-1)).quantile([lev_1, lev_2, lev_3, lev_4], dim='iter', skipna=True)

tmp_era5_wr3 = xr.open_mfdataset(
    '/glade/scratch/molina/s2s/bootstrap/t2m_era5_wr3_week12/t2m_era5_boot_*.nc',
    combine='nested', concat_dim='iter').chunk(
    dict(iter=-1)).quantile([lev_1, lev_2, lev_3, lev_4], dim='iter', skipna=True)

tmp_era5_wr4 = xr.open_mfdataset(
    '/glade/scratch/molina/s2s/bootstrap/t2m_era5_wr4_week12/t2m_era5_boot_*.nc',
    combine='nested', concat_dim='iter').chunk(
    dict(iter=-1)).quantile([lev_1, lev_2, lev_3, lev_4], dim='iter', skipna=True)

# cesm

tmp_cesm_wr1 = xr.open_mfdataset(
    '/glade/scratch/molina/s2s/bootstrap/t2m_cesm_wr1_week12/t2m_cesm_boot_*.nc',
    combine='nested', concat_dim='iter').chunk(
    dict(iter=-1)).quantile([lev_1, lev_2, lev_3, lev_4], dim='iter', skipna=True)

tmp_cesm_wr2 = xr.open_mfdataset(
    '/glade/scratch/molina/s2s/bootstrap/t2m_cesm_wr2_week12/t2m_cesm_boot_*.nc',
    combine='nested', concat_dim='iter').chunk(
    dict(iter=-1)).quantile([lev_1, lev_2, lev_3, lev_4], dim='iter', skipna=True)

tmp_cesm_wr3 = xr.open_mfdataset(
    '/glade/scratch/molina/s2s/bootstrap/t2m_cesm_wr3_week12/t2m_cesm_boot_*.nc',
    combine='nested', concat_dim='iter').chunk(
    dict(iter=-1)).quantile([lev_1, lev_2, lev_3, lev_4], dim='iter', skipna=True)

tmp_cesm_wr4 = xr.open_mfdataset(
    '/glade/scratch/molina/s2s/bootstrap/t2m_cesm_wr4_week12/t2m_cesm_boot_*.nc',
    combine='nested', concat_dim='iter').chunk(
    dict(iter=-1)).quantile([lev_1, lev_2, lev_3, lev_4], dim='iter', skipna=True)

## assemble dataset

In [18]:
ds_t2m = xr.Dataset(

    data_vars=dict(

        wr1_ncpc=(["lat", "lon"], t2m_noaa_01.unstack('new').isel(
            lead=slice(firstday, seconday)).stack(new=('lead', 'time')).mean(
            'new', skipna=True).where(mask == 1.0).values),
        wr2_ncpc=(["lat", "lon"], t2m_noaa_02.unstack('new').isel(
            lead=slice(firstday, seconday)).stack(new=('lead', 'time')).mean(
            'new', skipna=True).where(mask == 1.0).values),
        wr3_ncpc=(["lat", "lon"], t2m_noaa_03.unstack('new').isel(
            lead=slice(firstday, seconday)).stack(new=('lead', 'time')).mean(
            'new', skipna=True).where(mask == 1.0).values),
        wr4_ncpc=(["lat", "lon"], t2m_noaa_04.unstack('new').isel(
            lead=slice(firstday, seconday)).stack(new=('lead', 'time')).mean(
            'new', skipna=True).where(mask == 1.0).values),

        wr1_ncpc_025=(["lat", "lon"], tmp_ncpc_wr1.sel(
            quantile=0.025)['iteration'].transpose('lat', 'lon').values),
        wr1_ncpc_975=(["lat", "lon"], tmp_ncpc_wr1.sel(
            quantile=0.975)['iteration'].transpose('lat', 'lon').values),
        wr1_ncpc_005=(["lat", "lon"], tmp_ncpc_wr1.sel(
            quantile=0.005)['iteration'].transpose('lat', 'lon').values),
        wr1_ncpc_995=(["lat", "lon"], tmp_ncpc_wr1.sel(
            quantile=0.995)['iteration'].transpose('lat', 'lon').values),

        wr2_ncpc_025=(["lat", "lon"], tmp_ncpc_wr2.sel(
            quantile=0.025)['iteration'].transpose('lat', 'lon').values),
        wr2_ncpc_975=(["lat", "lon"], tmp_ncpc_wr2.sel(
            quantile=0.975)['iteration'].transpose('lat', 'lon').values),
        wr2_ncpc_005=(["lat", "lon"], tmp_ncpc_wr2.sel(
            quantile=0.005)['iteration'].transpose('lat', 'lon').values),
        wr2_ncpc_995=(["lat", "lon"], tmp_ncpc_wr2.sel(
            quantile=0.995)['iteration'].transpose('lat', 'lon').values),

        wr3_ncpc_025=(["lat", "lon"], tmp_ncpc_wr3.sel(
            quantile=0.025)['iteration'].transpose('lat', 'lon').values),
        wr3_ncpc_975=(["lat", "lon"], tmp_ncpc_wr3.sel(
            quantile=0.975)['iteration'].transpose('lat', 'lon').values),
        wr3_ncpc_005=(["lat", "lon"], tmp_ncpc_wr3.sel(
            quantile=0.005)['iteration'].transpose('lat', 'lon').values),
        wr3_ncpc_995=(["lat", "lon"], tmp_ncpc_wr3.sel(
            quantile=0.995)['iteration'].transpose('lat', 'lon').values),

        wr4_ncpc_025=(["lat", "lon"], tmp_ncpc_wr4.sel(
            quantile=0.025)['iteration'].transpose('lat', 'lon').values),
        wr4_ncpc_975=(["lat", "lon"], tmp_ncpc_wr4.sel(
            quantile=0.975)['iteration'].transpose('lat', 'lon').values),
        wr4_ncpc_005=(["lat", "lon"], tmp_ncpc_wr4.sel(
            quantile=0.005)['iteration'].transpose('lat', 'lon').values),
        wr4_ncpc_995=(["lat", "lon"], tmp_ncpc_wr4.sel(
            quantile=0.995)['iteration'].transpose('lat', 'lon').values),

        wr1_era5=(["lat", "lon"], t2m_era5_01.unstack('new').isel(
            lead=slice(firstday, seconday)).stack(new=('lead', 'time')).mean(
            'new', skipna=True).where(mask == 1.0).values),
        wr2_era5=(["lat", "lon"], t2m_era5_02.unstack('new').isel(
            lead=slice(firstday, seconday)).stack(new=('lead', 'time')).mean(
            'new', skipna=True).where(mask == 1.0).values),
        wr3_era5=(["lat", "lon"], t2m_era5_03.unstack('new').isel(
            lead=slice(firstday, seconday)).stack(new=('lead', 'time')).mean(
            'new', skipna=True).where(mask == 1.0).values),
        wr4_era5=(["lat", "lon"], t2m_era5_04.unstack('new').isel(
            lead=slice(firstday, seconday)).stack(new=('lead', 'time')).mean(
            'new', skipna=True).where(mask == 1.0).values),

        wr1_era5_025=(["lat", "lon"], tmp_era5_wr1.sel(
            quantile=0.025)['iteration'].transpose('lat', 'lon').values),
        wr1_era5_975=(["lat", "lon"], tmp_era5_wr1.sel(
            quantile=0.975)['iteration'].transpose('lat', 'lon').values),
        wr1_era5_005=(["lat", "lon"], tmp_era5_wr1.sel(
            quantile=0.005)['iteration'].transpose('lat', 'lon').values),
        wr1_era5_995=(["lat", "lon"], tmp_era5_wr1.sel(
            quantile=0.995)['iteration'].transpose('lat', 'lon').values),

        wr2_era5_025=(["lat", "lon"], tmp_era5_wr2.sel(
            quantile=0.025)['iteration'].transpose('lat', 'lon').values),
        wr2_era5_975=(["lat", "lon"], tmp_era5_wr2.sel(
            quantile=0.975)['iteration'].transpose('lat', 'lon').values),
        wr2_era5_005=(["lat", "lon"], tmp_era5_wr2.sel(
            quantile=0.005)['iteration'].transpose('lat', 'lon').values),
        wr2_era5_995=(["lat", "lon"], tmp_era5_wr2.sel(
            quantile=0.995)['iteration'].transpose('lat', 'lon').values),

        wr3_era5_025=(["lat", "lon"], tmp_era5_wr3.sel(
            quantile=0.025)['iteration'].transpose('lat', 'lon').values),
        wr3_era5_975=(["lat", "lon"], tmp_era5_wr3.sel(
            quantile=0.975)['iteration'].transpose('lat', 'lon').values),
        wr3_era5_005=(["lat", "lon"], tmp_era5_wr3.sel(
            quantile=0.005)['iteration'].transpose('lat', 'lon').values),
        wr3_era5_995=(["lat", "lon"], tmp_era5_wr3.sel(
            quantile=0.995)['iteration'].transpose('lat', 'lon').values),

        wr4_era5_025=(["lat", "lon"], tmp_era5_wr4.sel(
            quantile=0.025)['iteration'].transpose('lat', 'lon').values),
        wr4_era5_975=(["lat", "lon"], tmp_era5_wr4.sel(
            quantile=0.975)['iteration'].transpose('lat', 'lon').values),
        wr4_era5_005=(["lat", "lon"], tmp_era5_wr4.sel(
            quantile=0.005)['iteration'].transpose('lat', 'lon').values),
        wr4_era5_995=(["lat", "lon"], tmp_era5_wr4.sel(
            quantile=0.995)['iteration'].transpose('lat', 'lon').values),

        wr1_cesm=(["lat", "lon"], t2m_cesm_01.unstack('new').isel(lead=slice(
            firstday, seconday)).stack(new=('lead', 'time')).mean(
            'new', skipna=True).where(mask == 1.0).values),
        wr2_cesm=(["lat", "lon"], t2m_cesm_02.unstack('new').isel(lead=slice(
            firstday, seconday)).stack(new=('lead', 'time')).mean(
            'new', skipna=True).where(mask == 1.0).values),
        wr3_cesm=(["lat", "lon"], t2m_cesm_03.unstack('new').isel(lead=slice(
            firstday, seconday)).stack(new=('lead', 'time')).mean(
            'new', skipna=True).where(mask == 1.0).values),
        wr4_cesm=(["lat", "lon"], t2m_cesm_04.unstack('new').isel(lead=slice(
            firstday, seconday)).stack(new=('lead', 'time')).mean(
            'new', skipna=True).where(mask == 1.0).values),

        wr1_cesm_025=(["lat", "lon"], tmp_cesm_wr1.sel(
            quantile=0.025)['iteration'].transpose('lat', 'lon').values),
        wr1_cesm_975=(["lat", "lon"], tmp_cesm_wr1.sel(
            quantile=0.975)['iteration'].transpose('lat', 'lon').values),
        wr1_cesm_005=(["lat", "lon"], tmp_cesm_wr1.sel(
            quantile=0.005)['iteration'].transpose('lat', 'lon').values),
        wr1_cesm_995=(["lat", "lon"], tmp_cesm_wr1.sel(
            quantile=0.995)['iteration'].transpose('lat', 'lon').values),

        wr2_cesm_025=(["lat", "lon"], tmp_cesm_wr2.sel(
            quantile=0.025)['iteration'].transpose('lat', 'lon').values),
        wr2_cesm_975=(["lat", "lon"], tmp_cesm_wr2.sel(
            quantile=0.975)['iteration'].transpose('lat', 'lon').values),
        wr2_cesm_005=(["lat", "lon"], tmp_cesm_wr2.sel(
            quantile=0.005)['iteration'].transpose('lat', 'lon').values),
        wr2_cesm_995=(["lat", "lon"], tmp_cesm_wr2.sel(
            quantile=0.995)['iteration'].transpose('lat', 'lon').values),

        wr3_cesm_025=(["lat", "lon"], tmp_cesm_wr3.sel(
            quantile=0.025)['iteration'].transpose('lat', 'lon').values),
        wr3_cesm_975=(["lat", "lon"], tmp_cesm_wr3.sel(
            quantile=0.975)['iteration'].transpose('lat', 'lon').values),
        wr3_cesm_005=(["lat", "lon"], tmp_cesm_wr3.sel(
            quantile=0.005)['iteration'].transpose('lat', 'lon').values),
        wr3_cesm_995=(["lat", "lon"], tmp_cesm_wr3.sel(
            quantile=0.995)['iteration'].transpose('lat', 'lon').values),

        wr4_cesm_025=(["lat", "lon"], tmp_cesm_wr4.sel(
            quantile=0.025)['iteration'].transpose('lat', 'lon').values),
        wr4_cesm_975=(["lat", "lon"], tmp_cesm_wr4.sel(
            quantile=0.975)['iteration'].transpose('lat', 'lon').values),
        wr4_cesm_005=(["lat", "lon"], tmp_cesm_wr4.sel(
            quantile=0.005)['iteration'].transpose('lat', 'lon').values),
        wr4_cesm_995=(["lat", "lon"], tmp_cesm_wr4.sel(
            quantile=0.995)['iteration'].transpose('lat', 'lon').values),
    ),

    coords=dict(
        lon=(["lon"], lons),
        lat=(["lat"], lats),
    ),

    attrs=dict(description="Figure data for weather regimes research."),
)

/glade/work/molina/miniconda3/envs/myenv-tfgpu/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1395: RuntimeWarning: All-NaN slice encountered
  result = np.apply_along_axis(_nanquantile_1d, axis, a, q,
/glade/work/molina/miniconda3/envs/myenv-tfgpu/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1395: RuntimeWarning: All-NaN slice encountered
  result = np.apply_along_axis(_nanquantile_1d, axis, a, q,
/glade/work/molina/miniconda3/envs/myenv-tfgpu/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1395: RuntimeWarning: All-NaN slice encountered
  result = np.apply_along_axis(_nanquantile_1d, axis, a, q,


## save file

In [19]:
ds_t2m.to_netcdf('/glade/scratch/molina/s2s/bootstrap/t2m_week12_wxregimes.nc')